In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train.shape

In [ ]:
plt.imshow(x_train[0])

Normalization

In [ ]:
mean = np.mean(x_train)
std = np.std(x_train)

In [ ]:
x_train = (x_train - mean) / (std + 1e-7)
x_test = (x_test - mean) / (std + 1+ 1e-7)

## Limpieza

In [ ]:
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255

In [ ]:
num_clases = len(np.unique(y_train))
y_train = to_categorical(y_train,num_clases)
y_test = to_categorical(y_test,num_clases)

In [ ]:
(x_train, x_valid) = x_train[5000:],x_train[:5000]
(y_train, y_valid) = y_train[5000:],y_train[:5000]

In [ ]:
print('x_train_shape:', x_train.shape)

print('train', x_train.shape[0])
print('valid', x_valid.shape[0])
print('test', x_test.shape[0])

In [ ]:
base_filters = 32
w_regularizers = 1e-4

In [ ]:
model = Sequential()

Primero le pasamos la cantidad de filtros, segundo el tamaño del kernel, 3ro el padding

In [ ]:
## 1ra Convolutional
model.add(Conv2D(base_filters, (3,3), padding = 'same', 
                 kernel_regularizers = regularizers.l2(w_regularizers), 
                 input_shape = x_train.shape[1:]))


Puedo pasarle como función de activación relu en los parámetros o puedo agregarlo como una nueva capa

In [ ]:
model.add(Activation('relu'))

In [ ]:
## 2 Convolutional

model.add(Conv2D(base_filters, (3,3), padding = 'same', 
                 kernel_regularizers = regularizers.l2(w_regularizers)))
model.add(Activation('relu'))
model.add(BatchNormalization())

Primer Maxpooling para reducir la complejidad del modelo

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

3ra Convolutional

In [ ]:
model.add(Conv2D(2*base_filters, (3,3), padding = 'same', 
                 kernel_regularizers = regularizers.l2(w_regularizers)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

4ta Convolution

In [ ]:
model.add(Conv2D(2*base_filters, (3,3), padding = 'same', 
                 kernel_regularizers = regularizers.l2(w_regularizers)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.3))

5ta Convolution

In [ ]:
model.add(Conv2D(4*base_filters, (3,3), padding = 'same', 
                 kernel_regularizers = regularizers.l2(w_regularizers)))
model.add(Activation('relu'))
model.add(BatchNormalization())


6ta Convolution

In [ ]:
model.add(Conv2D(4*base_filters, (3,3), padding = 'same', 
                 kernel_regularizers = regularizers.l2(w_regularizers)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

In [ ]:
Clasificación - Flatten

In [ ]:
model.add(Flatten())
model.add(Dense(10, activation= 'softmax'))
model.summary()

In [ ]:
datagen = ImageDataGenerator(rotation_range = 15, 
                             width_shift_range = 0.1, 
                             height_shift_range = 0.1,
                             horizontal_flip = True,
                             vertical_flip = True)

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
#model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
model.compile(loss= 'categorical_crossentropy', optimizer = optimizers.Adam(), metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, batch_size=32, epochs = 100, validation_data = (x_valid, y_valid),
          verbose = 2, shuffle = True )

ModelCheckpoint salva el modelo cada vez que mejora la métrica que monitoree, en este caso monito = 'val_accuracy'

In [ ]:
checkpoint = ModelCheckpoint('mi_mejor_modelo.hdf5',
                             save_best_only = True,
                               verbose = 1, monitor = 'val_accuracy')

En este caso se está generando imágenes con ImageDataGenerator con el set de entrenamiento y el batch_size = 128 define la cantidad de imágenes que va a estar generando por cada lote.
Por ejemplo:
    

In [ ]:
hist = model.fit(datagen.flow(x_train, y_train, batch_size = 128),
          callbacks = [checkpoint],
          steps_per_epoch = x_train.shape[0] // 128,
          epochs =  120,
          verbose = 2,
          shuffle =True,
          validation_data = (x_valid, y_valid)
          )

In [ ]:
# plt.plot(history.history['accuracy'], label = 'Train')
# plt.plot(history.history['val_accuracy'], label = 'Validation')
# plt.legend()
# plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label = 'Train')
plt.plot(hist.history['val_accuracy'], label = 'Validation')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model2 = model

In [ ]:
model2.load_weights('./mi_mejor_modelo.hdf5')

In [ ]:
model2.evaluate(x_test, y_test)